In [1]:
import core
import synchronizer
import plot


In [2]:
path = "Y:\\Pietro\\2Precordings\\PM_211222_SRO004250\\rec_000\\suite2p\\plane0"

# Create sync object
sync = synchronizer.Sync(path+"\\rec_000_000.mat", path+"\\stim_sequence.npy", ["800-0","800-330","800-128","800-203","800-525"])
rec = core.Rec2P(path, sync)
cells = rec.get_cells(keep_unresponsive=True)



> loading data from Y:\Pietro\2Precordings\PM_211222_SRO004250\rec_000\suite2p\plane0 ... OK
> Using the configuration file found in data_path.
> parameters loaded.

> Extracting cells ...


100%|██████████| 1523/1523 [00:45<00:00, 33.41it/s]

> 414 responsive cells found (tot: 1523, keep_unresponsive: True)


In [ ]:
# refilter cells

# rec.update_params()

# for cell in cells:

#     cells[cell].is_responsive(qi_threshold=0.3)

pops = rec.get_populations("CONTRA",n_clusters=None, stims=["800-0"])

len(rec.get_responsive())

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.signal import butter, filtfilt

# rec.update_params()

# refilter cells
# for cell in cells:

#     cells[cell].is_responsive(qi_threshold=0.7)

pops_mean_resp = []

b, a = butter(4, 0.3)

for pop in pops:

    all_mean_resp = []

    for cell in pop:

        if cells[cell].responsive:

            concat_stims = np.array([])

            # concatenate the mean responses to all the stimuli

            for stim in cells[cell].analyzed_trials:

                for trial_type in cells[cell].analyzed_trials[stim]:

                    r = cells[cell].analyzed_trials[stim][trial_type]["average_dff"]

                    r = (r - r.min()) / (r.max()-r.min())
                    # r = np.where(r>r.mean(),1,0)
                    
                    concat_stims = np.concatenate((concat_stims,r))
        
        all_mean_resp.append(concat_stims)

    # convert to array
    all_mean_resp = np.array(all_mean_resp)

    plt.figure()
    plot.plot_heatmap(all_mean_resp, 15.5)
    plt.show()

    pops_mean_resp.append(all_mean_resp)

len(rec.get_responsive())

In [3]:
# refilter cells
for cell in cells:

    cells[cell].is_responsive(qi_threshold=0.6)


print(len(rec.get_responsive()))
    
plot.plot_averaged_trials(rec,path+"\\plots")

'Y:\\Pietro\\2Precordings\\PM_211222_SRO004250\\rec_000\\suite2p\\plane0\\plots\\average_responses_dff'